In [1]:
# 재시작 없이 import 파일 자동 적용
%load_ext autoreload
%autoreload 2

In [2]:
import os
import h5py
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import matthews_corrcoef
import torch
from torch.utils.data import DataLoader

# from config import CONFIG
from data_factory import collate_fn
from data_loader import PublicTest
from classification import Exp_Classification

In [254]:
torch.cuda.empty_cache()

### Checkpoints 및 CONFIG file 경로 설정

In [3]:
import importlib.util

test_folder_name = 'classification_T#27_Medformer_K-Medicon_bs4_250hz_CrossEntropy_sl96_lr0.0001_pl96_dm128_nh8_el6_df256_fc1_ebtimeF_relu_Exp_seed41_jitter0.2,scale0.2,drop0.5'
test_config_path = f"C:/Git/Artifact-Detection-In-ECG/T27/checkpoints/{test_folder_name}/config_T#27.py"

# 모듈 이름 생성
module_name = os.path.splitext(os.path.basename(test_config_path))[0]

# 모듈 로드
spec = importlib.util.spec_from_file_location(module_name, test_config_path)
config_module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(config_module)

# CONFIG 변수 사용
CONFIG = config_module.CONFIG  # config_T#01.py 파일 내의 CONFIG 변수

In [4]:
print(CONFIG)

{'seed': 41, 'is_training': 1, 'model_id': 'T#27', 'task_name': 'classification', 'model': 'Medformer', 'monitor': 'vali_loss', 'data': 'K-Medicon', 'root_path': './dataset/', 'signal_hz': 250, 'seq_len': 96, 'pred_len': 96, 'enc_in': 7, 'c_out': 7, 'd_model': 128, 'n_heads': 8, 'e_layers': 6, 'd_ff': 256, 'factor': 1, 'dropout': 0.1, 'embed': 'timeF', 'activation': 'relu', 'output_attention': False, 'no_inter_attn': False, 'patch_len_list': '2,4,8,8,16,16,16,16,32,32,32,32,32,32,32,32', 'single_channel': False, 'augmentations': 'jitter0.2,scale0.2,drop0.5', 'num_workers': 0, 'itr': 1, 'train_epochs': 100, 'batch_size': 4, 'patience': 5, 'learning_rate': 0.0001, 'des': 'Exp', 'loss': 'CrossEntropy', 'swa': True, 'use_gpu': True, 'gpu': 0, 'use_multi_gpu': False, 'devices': '0,1,2,3'}


In [5]:
seed = CONFIG['seed']
    
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [6]:
setting = "{}_{}_{}_{}_bs{}_{}hz_{}_sl{}_lr{}_pl{}_dm{}_nh{}_el{}_df{}_fc{}_eb{}_{}_{}_seed{}_{}".format(
                CONFIG['task_name'],
                CONFIG['model_id'],
                CONFIG['model'],
                CONFIG['data'],
                CONFIG['batch_size'],
                CONFIG['signal_hz'],
                CONFIG['loss'],
                CONFIG['seq_len'],
                CONFIG['learning_rate'],
                CONFIG['pred_len'],
                CONFIG['d_model'],
                CONFIG['n_heads'],
                CONFIG['e_layers'],
                CONFIG['d_ff'],
                CONFIG['factor'],
                CONFIG['embed'],
                CONFIG['activation'],
                CONFIG['des'],
                CONFIG['seed'],
                CONFIG['augmentations'],
            )

exp = Exp_Classification(CONFIG)

Use GPU: cuda:0


In [7]:
model_path = f"./checkpoints/{test_folder_name}/1004_checkpoint.pth"

exp.swa_model.load_state_dict(torch.load(model_path))
exp.swa_model.eval()

AveragedModel(
  (module): Model(
    (enc_embedding): ListPatchEmbedding(
      (value_embeddings): ModuleList(
        (0): CrossChannelTokenEmbedding(
          (tokenConv): Conv2d(1, 128, kernel_size=(1, 2), stride=(1, 2), bias=False, padding_mode=circular)
        )
        (1): CrossChannelTokenEmbedding(
          (tokenConv): Conv2d(1, 128, kernel_size=(1, 4), stride=(1, 4), bias=False, padding_mode=circular)
        )
        (2-3): 2 x CrossChannelTokenEmbedding(
          (tokenConv): Conv2d(1, 128, kernel_size=(1, 8), stride=(1, 8), bias=False, padding_mode=circular)
        )
        (4-7): 4 x CrossChannelTokenEmbedding(
          (tokenConv): Conv2d(1, 128, kernel_size=(1, 16), stride=(1, 16), bias=False, padding_mode=circular)
        )
        (8-15): 8 x CrossChannelTokenEmbedding(
          (tokenConv): Conv2d(1, 128, kernel_size=(1, 32), stride=(1, 32), bias=False, padding_mode=circular)
        )
      )
      (position_embedding): PositionalEmbedding()
      (drop

---
### 250 hz

In [8]:
# Public label
testset = PublicTest("./dataset/normalized_sigle_lead_Signal_Test_Public.h5")
public_test_loader = DataLoader(
    testset,
    batch_size=1,
    shuffle=False,
    num_workers=0,
    drop_last=False
    )

In [9]:
device = torch.device("cuda:0")

total_loss = []
preds = []
trues = []

with torch.no_grad():
    for i, (batch_x, label) in enumerate(tqdm(public_test_loader)):
        
        downsample_factor = 4
        downsampled_array = batch_x.reshape(1, -1, downsample_factor, 1).mean(axis=2)
        batch_x = downsampled_array
        
        batch_x = batch_x.float().to(device)
        label = label.to(device)
        # padding_mask = padding_mask.float().to(device)

        outputs = exp.swa_model(batch_x, None, None, None)

        pred = outputs.detach().cpu()

        preds.append(outputs.detach())
        trues.append(label)

preds = torch.cat(preds, 0)
trues = torch.cat(trues, 0)

probs = torch.nn.functional.softmax(
    preds
)  # (total_samples, num_classes) est. prob. for each class and sample

trues_onehot = (torch.nn.functional.one_hot(trues.reshape(-1,).to(torch.long),
                num_classes=CONFIG['num_class'],).float().cpu().numpy())

print(trues_onehot.shape)
predictions = (
    torch.argmax(probs, dim=1).cpu().numpy()
)  # (total_samples,) int class index for each sample
probs = probs.cpu().numpy()
trues = trues.flatten().cpu().numpy()

100%|██████████| 300/300 [01:36<00:00,  3.12it/s]

(300, 2)


In [10]:
f1 = f1_score(trues, predictions, average="binary")
auroc = roc_auc_score(trues_onehot, probs)
# auroc = roc_auc_score(trues, probs)
mcc = matthews_corrcoef(trues, predictions)
cpi = (0.25 * f1) + (0.25 * auroc) + (0.5 * mcc)

print(f"F1 score: {f1}")
print(f"AUROC score: {auroc}")
print(f"MCC score: {mcc}")
print(f"CPI score: {cpi}")
print()
print(f"0개수: {len(np.where(predictions[:]==0)[0])}")
print(f"1개수: {len(np.where(predictions[:]==1)[0])}")

F1 score: 0.6357615894039735
AUROC score: 0.8590222222222222
MCC score: 0.5132944359184718
CPI score: 0.6303431708657848

0개수: 224
1개수: 76


In [11]:
# df = pd.DataFrame({"0":probs[:,0], "1":probs[:,1]})
df = pd.DataFrame({"Probability":probs[:,1]})

save_xlsx = pd.ExcelWriter("./RTFACT_241004_Public_Possibility_JT27.xlsx")
df.to_excel(save_xlsx, index = False) # xlsx 파일로 변환
save_xlsx.save() #xlsx 파일로 저장

In [12]:
predictions[15]

1

In [265]:
# Public jamm label
testset = PublicTest("./dataset/Signal_Test_Public_250hz_jamm.h5")
public_test_loader = DataLoader(
    testset,
    batch_size=1,
    shuffle=False,
    num_workers=0,
    drop_last=False,
    # collate_fn=lambda x: collate_fn(
    #             x, max_len=CONFIG['seq_len']),
    )

In [266]:
device = torch.device("cuda:0")

total_loss = []
preds = []
trues = []

with torch.no_grad():
    for i, (batch_x, label) in enumerate(tqdm(public_test_loader)):
        batch_x = batch_x.float().to(device)
        label = label.to(device)
        # padding_mask = padding_mask.float().to(device)

        outputs = exp.swa_model(batch_x, None, None, None)

        pred = outputs.detach().cpu()

        preds.append(outputs.detach())
        trues.append(label)

preds = torch.cat(preds, 0)
trues = torch.cat(trues, 0)

probs = torch.nn.functional.softmax(
    preds
)  # (total_samples, num_classes) est. prob. for each class and sample

trues_onehot = (torch.nn.functional.one_hot(trues.reshape(-1,).to(torch.long),
                num_classes=CONFIG['num_class'],).float().cpu().numpy())

print(trues_onehot.shape)
predictions = (
    torch.argmax(probs, dim=1).cpu().numpy()
)  # (total_samples,) int class index for each sample
probs = probs.cpu().numpy()
trues = trues.flatten().cpu().numpy()

100%|██████████| 300/300 [00:11<00:00, 25.04it/s]

(300, 2)


In [267]:
f1 = f1_score(trues, predictions, average="binary")
auroc = roc_auc_score(trues_onehot, probs)
# auroc = roc_auc_score(trues, probs)
mcc = matthews_corrcoef(trues, predictions)
cpi = (0.25 * f1) + (0.25 * auroc) + (0.5 * mcc)

print('================== PUBLIC JAMM ==================')
print(f"F1 score: {f1}")
print(f"AUROC score: {auroc}")
print(f"MCC score: {mcc}")
print(f"CPI score: {cpi}")
print()
print(f"0개수: {len(np.where(predictions[:]==0)[0])}")
print(f"1개수: {len(np.where(predictions[:]==1)[0])}")

================== PUBLIC JAMM ==================
F1 score: 0.65625
AUROC score: 0.859437037037037
MCC score: 0.5802976362831692
CPI score: 0.6690705774008439

0개수: 247
1개수: 53


### 500 hz

In [133]:
# Public label (500hz)
testset = PublicTest("./dataset/Signal_Test_Public_500hz.h5")
public_test_loader = DataLoader(
    testset,
    batch_size=1,
    shuffle=False,
    num_workers=0,
    drop_last=False,
    # collate_fn=lambda x: collate_fn(
    #             x, max_len=CONFIG['seq_len']),
    )

In [135]:
# Public jamm label (500hz)
testset = PublicTest("./dataset/Signal_Test_Public_500hz_jamm.h5")
public_test_loader = DataLoader(
    testset,
    batch_size=1,
    shuffle=False,
    num_workers=0,
    drop_last=False,
    # collate_fn=lambda x: collate_fn(
    #             x, max_len=CONFIG['seq_len']),
    )

In [ ]:
device = torch.device("cuda:0")

total_loss = []
preds = []
trues = []

with torch.no_grad():
    for i, (batch_x, label) in enumerate(tqdm(public_test_loader)):
        batch_x = batch_x.float().to(device)
        label = label.to(device)
        # padding_mask = padding_mask.float().to(device)

        outputs = exp.swa_model(batch_x, None, None, None)

        pred = outputs.detach().cpu()

        preds.append(outputs.detach())
        trues.append(label)

preds = torch.cat(preds, 0)
trues = torch.cat(trues, 0)

probs = torch.nn.functional.softmax(
    preds
)  # (total_samples, num_classes) est. prob. for each class and sample

trues_onehot = (torch.nn.functional.one_hot(trues.reshape(-1,).to(torch.long),
                num_classes=CONFIG['num_class'],).float().cpu().numpy())

print(trues_onehot.shape)
predictions = (
    torch.argmax(probs, dim=1).cpu().numpy()
)  # (total_samples,) int class index for each sample
probs = probs.cpu().numpy()
trues = trues.flatten().cpu().numpy()

In [ ]:
f1 = f1_score(trues, predictions, average="binary")
auroc = roc_auc_score(trues_onehot, probs)
# auroc = roc_auc_score(trues, probs)
mcc = matthews_corrcoef(trues, predictions)
cpi = (0.25 * f1) + (0.25 * auroc) + (0.5 * mcc)

print(f"F1 score: {f1}")
print(f"AUROC score: {auroc}")
print(f"MCC score: {mcc}")
print(f"CPI score: {cpi}")
print()
print(f"0개수: {len(np.where(predictions[:]==0)[0])}")
print(f"1개수: {len(np.where(predictions[:]==1)[0])}")

In [ ]:
# df = pd.DataFrame({"0":probs[:,0], "1":probs[:,1]})
df = pd.DataFrame({"Probability":probs[:,1]})

save_xlsx = pd.ExcelWriter(f"./RTFACT_241004_Public_Possibility_{CONFIG['model_id']}.xlsx")
df.to_excel(save_xlsx, index = False) # xlsx 파일로 변환
save_xlsx.save() #xlsx 파일로 저장

In [ ]:
predictions[15]